In [1]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import json
import string
import time,os
import numpy as np
import pandas as pd
import re

import requests
import time, os

from selenium import webdriver
from selenium.webdriver import Chrome
from selenium.webdriver.common.keys import Keys

In [2]:
driver_path="C:\\Users\\easan\\chromedriver.exe"
driver=webdriver.Chrome(driver_path)

In [3]:
url="https://www.hurriyetemlak.com/ankara-satilik"
driver.get(url)
r=requests.get(url)
if r.status_code == 429:
    time.sleep(int(r.headers["Retry-After"]))
r.status_code

200

In [4]:
soup=BeautifulSoup(r.content,"lxml")
pages=soup.find_all("ul",{"class":"pagination"})[0].find_all("li")
page_count=int(pages[-2].text) #sonraki sayfadan bir önce olan tüm sayfa sayısıdır

In [5]:
page_count

1401

In [6]:
props_dict = {'listing_id':[],'price':[],'city':[],'district':[],'neighborhood':[],'last_edit': [],'listing_status':[],'housing_type':[],'number_of_rooms':[], 'area': [], 'prop_floor': [], 'building_age': [], 'heating_type': [],
              'building_floors': [], 'credit_availability': [], 'furniture_availability': [], 'bathrooms': [],
              'building_type': [], 'building_status': [], 'usage_status': [], 'deed': [], 'dues': [], 'swap': [],
               'facade': [], 'fuel_type': [],'inside_the_site':[],'rental_income':[],'title':[]}

In [7]:
def det_title_upper_func(det_title_upper):
    price='None'
    title='None'
    
    price=det_title_upper.find("p",{"class":"price"}).text.strip()
    title=det_title_upper.find("h1").text.strip()
    
    props_dict['price'].append(price) 
    props_dict['title'].append(title) 

In [8]:
def short_info_list_func(short_info_list):
    city = 'None'
    district = 'None'
    neighborhood = 'None'
    area = 'None'

    i=1
        
    for li in short_info_list:
        if(i==1):
            city = li.text.strip()
        elif(i==2):
            district = li.text.strip()
        elif(i==3):
            neighborhood = li.text.strip()
        elif(i==7):
            area = li.text.strip()
        i = i + 1
    
    props_dict['city'].append(city)        
    props_dict['district'].append(district)
    props_dict['neighborhood'].append(neighborhood)
    props_dict['area'].append(area)

In [9]:
def add_info_list_func(add_info_list):
    listing_id= 'None'
    last_edit = 'None'
    listing_status='None'
    housing_type='None'
    number_of_rooms='None'
    prop_floor = 'None'
    building_age = 'None'
    heating_type = 'None'
    building_floors = 'None'
    credit_availability = 'None'
    furniture_availability = 'None'
    bathrooms = 'None'
    building_type = 'None'
    building_status = 'None'
    usage_status = 'None'
    deed = 'None'
    dues = 'None'
    swap = 'None'
    student_single = 'None'
    facade = 'None'
    fuel_type = 'None'
    inside_the_site='None'
    rental_income='None'


        
    for li in add_info_list:
        param=li.find_all("span")
        param_type=param[0].text.strip()
        param_value=param[1].text.strip()
        
        if(param_type=='İlan no'):
            listing_id = param_value
        elif(param_type=='Son Güncelleme Tarihi'):
            last_edit = param_value
        elif(param_type=='İlan Durumu'):
            listing_status = param_value
        elif(param_type=='Konut Şekli'):
            housing_type = param_value
        elif(param_type=='Oda + Salon Sayısı'):
            number_of_rooms = param_value
        elif(param_type=='Bulunduğu Kat'):
            prop_floor = param_value
        elif(param_type=='Bina Yaşı'):
            building_age = param_value
        elif(param_type=='Isınma Tipi'):
            heating_type = param_value
        elif(param_type=='Kat Sayısı'):
            building_floors = param_value
        elif(param_type=='Krediye Uygunluk'):
            credit_availability = param_value
        elif(param_type=='Eşya Durumu'):
            furniture_availability = param_value
        elif(param_type=='Banyo Sayısı'):
            bathrooms = param_value
        elif(param_type=='Yapı Tipi'):
            building_type = param_value
        elif(param_type=='Yapının Durumu'):
            building_status = param_value
        elif(param_type=='Kullanım Durumu'):
            usage_status = param_value
        elif(param_type=='Tapu Durumu'):
            deed = param_value
        elif(param_type=='Aidat'):
            dues = param_value
        elif(param_type=='Takas'):
            swap = param_value
        elif(param_type=='Cephe'):
            facade = param_value
        elif(param_type=='Site İçerisinde'):
            inside_the_site = param_value
        elif(param_type=='Kira Getirisi'):
            rental_income = param_value
        elif(param_type=='Yakıt Tipi'):
            fuel_type = param_value
            
    props_dict['listing_id'].append(listing_id)        
    props_dict['last_edit'].append(last_edit)
    props_dict['listing_status'].append(listing_status)
    props_dict['housing_type'].append(housing_type)
    props_dict['number_of_rooms'].append(number_of_rooms)
    props_dict['prop_floor'].append(prop_floor)
    props_dict['building_age'].append(building_age)
    props_dict['heating_type'].append(heating_type)
    props_dict['building_floors'].append(building_floors)
    props_dict['credit_availability'].append(credit_availability)
    props_dict['furniture_availability'].append(furniture_availability)
    props_dict['bathrooms'].append(bathrooms)
    props_dict['building_type'].append(building_type)
    props_dict['building_status'].append(building_status)
    props_dict['usage_status'].append(usage_status)
    props_dict['deed'].append(deed)
    props_dict['dues'].append(dues)
    props_dict['swap'].append(swap)
    props_dict['facade'].append(facade)
    props_dict['fuel_type'].append(fuel_type)
    props_dict['inside_the_site'].append(inside_the_site)
    props_dict['rental_income'].append(rental_income)
    

In [10]:


for page in range(1025,1052):#page_count+1
    
    url_home_page=url+"?page={}".format(page)
    driver=webdriver.Chrome(driver_path)
    driver.get(url_home_page)
    
    pageRequest=requests.get(url+"?page={}".format(page),headers = {'User-agent': 'your bot 0.1'})
    
    if pageRequest.status_code == 429:
          time.sleep(int(pageRequest.headers["Retry-After"]))
            
    #print(pageRequest.url)
    pageSoup=BeautifulSoup(pageRequest.content,"lxml")
    links=pageSoup.findAll("a",{"class":"card-link"})
    for link in links:
        
        homeLink="https://www.hurriyetemlak.com"+link.get("href")

        homeR=requests.get(homeLink,headers = {'User-agent': 'your bot 0.1'})
        homeR.status_code
        homeSoup=BeautifulSoup(homeR.content,"lxml")
        
        home_features=homeSoup.find("section",{"class":"det-block"})

        add_info_list=home_features.find("ul",{"class":"adv-info-list"}).find_all("li")
        short_info_list=home_features.find("ul",{"class":"short-info-list"}).find_all("li")
        det_title_upper=home_features.find("div",{"class":"det-title-upper"})
        
        
        add_info_list_func(add_info_list)
        short_info_list_func(short_info_list)
        det_title_upper_func(det_title_upper)
        
        
driver.close()

In [11]:
for key in props_dict.keys():
    print('name:{} size: {}'.format(key,len(props_dict[key])))

name:listing_id size: 648
name:price size: 648
name:city size: 648
name:district size: 648
name:neighborhood size: 648
name:last_edit size: 648
name:listing_status size: 648
name:housing_type size: 648
name:number_of_rooms size: 648
name:area size: 648
name:prop_floor size: 648
name:building_age size: 648
name:heating_type size: 648
name:building_floors size: 648
name:credit_availability size: 648
name:furniture_availability size: 648
name:bathrooms size: 648
name:building_type size: 648
name:building_status size: 648
name:usage_status size: 648
name:deed size: 648
name:dues size: 648
name:swap size: 648
name:facade size: 648
name:fuel_type size: 648
name:inside_the_site size: 648
name:rental_income size: 648
name:title size: 648


In [12]:
df=pd.DataFrame(props_dict)

In [13]:
df.to_csv("hurriyetEmlak14.csv")